In [1]:
data = [
    {
        'passport_div_code': 422008,
        'passport_issuer_name': 'БЕЛОВСКИМ УВД КЕМЕРОВСКОЙ ОБЛАСТИ',
    }, 
    {
        'passport_div_code': 500112,
        'passport_issuer_name': 'ТП №2 В ГОР. ОРЕХОВО-ЗУЕВО ОУФМС РОССИИ ПО МО',
    }, 
    {
        'passport_div_code': 642001,
        'passport_issuer_name': 'ВОЛЖСКИМ РОВД ГОР.САРАТОВА',
    }, 
    {
        'passport_div_code': 162004,
        'passport_issuer_name': 'УВД МОСКОВСКОГО РАЙОНА Г.КАЗАНЬ',
    }, 
    {
        'passport_div_code': 80001,
        'passport_issuer_name': 'ОТДЕЛОМ ОФМС РОССИИ ПО РЕСП КАЛМЫКИЯ В Г ЭЛИСТА',
    }
]

In [2]:
import pandas as pd

In [3]:
train = pd.DataFrame(data)

In [4]:
train.passport_issuer_name = train.passport_issuer_name.str.lower()

In [5]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'р-(а|й|о|н|е)*',u'район')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' г( |\.|(ор(\.| )))', u' город ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' р(\.|есп )', u' республика ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' адм([а-я]*)(\.)?', u' административный ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' окр(\.| |уга( )?)', u' округ ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' ао ', u' административный округ ')

C:\Users\Daniil\AppData\Local\Temp/ipykernel_19740/4068230019.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train.passport_issuer_name = train.passport_issuer_name.str.replace(u'р-(а|й|о|н|е)*',u'район')
C:\Users\Daniil\AppData\Local\Temp/ipykernel_19740/4068230019.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train.passport_issuer_name = train.passport_issuer_name.str.replace(u' г( |\.|(ор(\.| )))', u' город ')
C:\Users\Daniil\AppData\Local\Temp/ipykernel_19740/4068230019.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train.passport_issuer_name = train.passport_issuer_name.str.replace(u' р(\.|есп )', u' республика ')
C:\Users\Daniil\AppData\Local\Temp/ipykernel_19740/4068230019.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train.passport_issuer_name = train.passport

In [6]:
train.passport_issuer_name = train.passport_issuer_name.str.replace(u' - ?', u'-')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'[^а-я -]','')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'- ',' ')
train.passport_issuer_name = train.passport_issuer_name.str.replace(u'  *',' ')

C:\Users\Daniil\AppData\Local\Temp/ipykernel_19740/899739293.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train.passport_issuer_name = train.passport_issuer_name.str.replace(u' - ?', u'-')
C:\Users\Daniil\AppData\Local\Temp/ipykernel_19740/899739293.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train.passport_issuer_name = train.passport_issuer_name.str.replace(u'[^а-я -]','')
C:\Users\Daniil\AppData\Local\Temp/ipykernel_19740/899739293.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  train.passport_issuer_name = train.passport_issuer_name.str.replace(u'  *',' ')


In [7]:
train

,passport_div_code,passport_issuer_name
0,422008,беловским увд кемеровской области
1,500112,тп в город орехово-зуево оуфмс россии по мо
2,642001,волжским ровд город саратова
3,162004,увд московского района город казань
4,80001,отделом офмс россии по республика калмыкия в г...


In [9]:
df = train.copy()

In [14]:
df

,passport_div_code,passport_issuer_name
0,422008,беловским увд кемеровской области
1,500112,тп в город орехово-зуево оуфмс россии по мо
2,642001,волжским ровд город саратова
3,162004,увд московского района город казань
4,80001,отделом офмс россии по республика калмыкия в г...


In [15]:
import texthero as hero
from texthero import preprocessing
custom_pipeline = [preprocessing.fillna,
                   #preprocessing.lowercase,
                   preprocessing.remove_whitespace,
                   preprocessing.remove_diacritics
                   #preprocessing.remove_brackets
                  ]
df['clean_text'] = hero.clean(df['passport_issuer_name'], custom_pipeline)
df['clean_text'] = [n.replace('{','') for n in df['clean_text']]
df['clean_text'] = [n.replace('}','') for n in df['clean_text']]
df['clean_text'] = [n.replace('(','') for n in df['clean_text']]
df['clean_text'] = [n.replace(')','') for n in df['clean_text']]

In [16]:
df

,passport_div_code,passport_issuer_name,clean_text
0,422008,беловским увд кемеровской области,belovskim uvd kemerovskoi oblasti
1,500112,тп в город орехово-зуево оуфмс россии по мо,tp v gorod orekhovo-zuevo oufms rossii po mo
2,642001,волжским ровд город саратова,volzhskim rovd gorod saratova
3,162004,увд московского района город казань,uvd moskovskogo raiona gorod kazan'
4,80001,отделом офмс россии по республика калмыкия в г...,otdelom ofms rossii po respublika kalmykiia v ...


In [21]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

card_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df.clean_text)]

card_docs

[TaggedDocument(words=['belovskim', 'uvd', 'kemerovskoi', 'oblasti'], tags=[0]),
 TaggedDocument(words=['tp', 'v', 'gorod', 'orekhovo-zuevo', 'oufms', 'rossii', 'po', 'mo'], tags=[1]),
 TaggedDocument(words=['volzhskim', 'rovd', 'gorod', 'saratova'], tags=[2]),
 TaggedDocument(words=['uvd', 'moskovskogo', 'raiona', 'gorod', "kazan'"], tags=[3]),
 TaggedDocument(words=['otdelom', 'ofms', 'rossii', 'po', 'respublika', 'kalmykiia', 'v', 'gorod', 'elista'], tags=[4])]

In [23]:
model = Doc2Vec(vector_size=16, min_count=1, epochs = 20)

In [24]:
model = Doc2Vec(vector_size=16, window=2, min_count=1, workers=8, epochs=20)

model.build_vocab(card_docs)

model.train(card_docs, total_examples=model.corpus_count
            , epochs=model.epochs)

In [25]:
card2vec = [model.infer_vector((df['clean_text'][i].split(' '))) 
            for i in range(0,len(df['clean_text']))]

In [27]:
import numpy as np

dtv= np.array(card2vec).tolist()

df['card2vec'] = dtv

In [28]:
df['tsnec2v'] = hero.tsne(df['card2vec'])

In [29]:
df

,passport_div_code,passport_issuer_name,clean_text,card2vec,tsnec2v
0,422008,беловским увд кемеровской области,belovskim uvd kemerovskoi oblasti,"[-0.018718330189585686, -0.005538497120141983,...","[-224.20648193359375, 22.09152603149414]"
1,500112,тп в город орехово-зуево оуфмс россии по мо,tp v gorod orekhovo-zuevo oufms rossii po mo,"[0.014958590269088745, -0.005290632601827383, ...","[341.5960693359375, 249.90988159179688]"
2,642001,волжским ровд город саратова,volzhskim rovd gorod saratova,"[-0.030248921364545822, -0.02887367084622383, ...","[319.6063232421875, -82.45893859863281]"
3,162004,увд московского района город казань,uvd moskovskogo raiona gorod kazan',"[-0.004950718954205513, -0.021067308261990547,...","[20.118270874023438, 231.78504943847656]"
4,80001,отделом офмс россии по республика калмыкия в г...,otdelom ofms rossii po respublika kalmykiia v ...,"[-0.017142238095402718, 0.021305445581674576, ...","[21.82888412475586, -202.46124267578125]"
